In [1]:
%run ../00_default_options.ipynb

In [2]:
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

In [3]:
cohort = Cohort.load(id=1)

In [4]:
onset_dataframe = OnsetDataFrame.load(id=17)

In [5]:
feature_pipeline = FeaturePipeline.load(id=19)

In [6]:
import morpher
from morpher.jobs import *

Using TensorFlow backend.


In [7]:
agg_func_regex = 'any__'

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]] = label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]].fillna(False)
    
    imputed_data, _ = Impute().execute(data=label_encoded_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [8]:
from itertools import product

In [9]:
thresholds_numeric = [(x / 100) for x in range(5, 100, 5)]
observation_windows_numeric = [[x, -1] for x in range(-361, -1, 30)]
thresholds_occurring = [(x / 100) for x in range(5, 100, 5)]
observation_windows_occurring = [[x, -1] for x in range(-361, -1, 30)]

In [10]:
iterator = product(
    thresholds_numeric,
    observation_windows_numeric,
    thresholds_occurring,
    observation_windows_occurring
)

configs = [x for x in iterator if x[0] == x[2] and x[1] == x[3]]
targets = ['urinary_tract_infection_onset_from_0_days_after_to_365_days_after']
algorithms = [morpher.config.algorithms.GBDT, morpher.config.algorithms.RF, morpher.config.algorithms.DT, morpher.config.algorithms.LR]
samplers = samplers = [morpher.config.samplers.RANDOM, morpher.config.samplers.URANDOM, morpher.config.samplers.SMOTE, morpher.config.samplers.NOSAMPLER]

In [11]:
pipeline = training.execute(
    comment='baseline-1 UTI',
    version='11.0.0',
    cohort=cohort,
    onset_dataframe=onset_dataframe,
    feature_pipeline=feature_pipeline,
    data_loader=data_loader,
    observation_iterator=configs,
    targets=targets,
    algorithms=algorithms,
    samplers=samplers,
    feature_type_occurring="occurring",
    feature_type_numeric="numeric",
    rfe__run=False,
    rfe__step_size=50
)

Columns (11,18,19,20,24,35,64,83,90,92,94,96,97,106,114,117,120,125,126,133,136,140,141,142,143,144,150,178,235,244,248,288,311,317,320,325,326,330,334,341,353,359,360,361,365,402,421,425,430,434,436,439,446) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f98a65c82e8>
Fetching data for Patient (...)



*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 359.043ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 895.145ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 570.038ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 376.993ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 516.162ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 27.509ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 205.758ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 857.9ms

*** Training of model 'GradientBoostingClassifier'

lbfgs failed to converge. Increase the number of iterations.


*** Training of classifier ready. Time elapsed: 186.088ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 75.382ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 41.678ms

*** Training of model 'LogisticRegression' started.


...

F-score is ill-defined and being set to 0.0 due to no predicted samples.
Precision is ill-defined and being set to 0.0 due to no predicted samples.
invalid value encountered in long_scalars


*** Training of classifier ready. Time elapsed: 120.977ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 78.643ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 127.028ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 64.343ms



F-score is ill-defined and being set to 0.0 due to no predicted samples.
Precision is ill-defined and being set to 0.0 due to no predicted samples.
invalid value encountered in long_scalars
F-score is ill-defined and being set to 0.0 due to no predicted samples.
Precision is ill-defined and being set to 0.0 due to no predicted samples.
invalid value encountered in long_scalars


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f98a591d5f8>
Fetching data for Patient (...)
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 301.066ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 317.011ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 332.705ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 330.048ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 347.471ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 204.133ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 200.605ms

*** Training of model 'GradientBoostingClassifier' started.
*** 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
pipeline.id

234